# SubPTJ crawling

## Crawling

In [1]:
import time
import json
import urllib
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

### HRD-net URL
---
한 페이지에서 원하는 데이터 뽑아서 모든 페이지 적용할 예정

In [2]:
url = 'https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20200000286660&tracseTme=1&crseTracseSe=C0061&trainstCstmrId=500020018488#undefined'

resp = requests.get(url)
soup = BeautifulSoup(resp.text, 'html.parser')

> 기관명

In [3]:
# 기관명
for organ in soup.select('p.add'):
    print(organ.text.strip())
#     data.append(organ.text.strip())

스마트인재개발원


> 과정명

In [4]:
# 과정명
for course in soup.select('h4.tit'):
    print(course.text.replace('\n','').replace('\t',''))
#     data.append(course.text.replace('\n','').replace('\t',''))

빅데이터 분석서비스 개발자과정(NCS)모집마감


> 훈련비

In [5]:
# 훈련비
for price in soup.select('p.price'):
    print(price.text.strip().replace('\n','')[:price.text.find('원')].strip().replace('\t',''))
#     data.append(price.text.strip().replace('\n','')[:price.text.find('원')].strip().replace('\t',''))

13,762,240 원



In [6]:

for i in soup.select('span.con'):
    if i.text.find('일') != -1:
        date = i.text.replace('\n,','').replace('\t','').strip().replace('\n','')
        print(f'date : {date}')
    elif i.text.find('세') != -1:
        age = i.text.replace('\n,','').replace('\t','').strip()
        print('age : ' + age)
    elif i.text.find(')') != -1:
        if  i.text.find('~') == -1:
            NCS = i.text.replace('\n,','').replace('\t','').strip()
            print('NCS : ' + NCS[:NCS.find('(')])
        else:
            print('datetime : ' + i.text.replace('\n,','').replace('\t','').strip())
    
#         data.append(i.text.replace('\n,','').replace('\t','').strip())
#     print(i.text.replace('\n', '').replace('\t','').strip())

NCS : 응용SW엔지니어링
datetime : 2020-11-02
~2021-04-12
(1회차)
date : 116일,총928시간
age : 27 세


In [7]:
# NCS 자격분류
for NCS in soup.select('span.con'):
    if re.findall('\d{6,}',NCS.text) != []:
        print(re.findall('.+\d{6,}.',NCS.text).pop().replace('\t',''))

응용SW엔지니어링(20010202)


> span 태그가 적절하지 않음 -> 쓸데없는게 너무 뽑힘

> 근데 대안도 없다 이걸로 가자

In [8]:
# 수강생 평균 만족도
soup.select('div.starRating')[0].get('aria-label')

'만족도 100점 기준86.88점'

In [9]:
# 훈련시간
for i in soup.select('span.con'):
    if i.text.find('일') != -1:
        print(i.text.replace('\n,','').replace('\t','').strip())
#         data.append(i.text.replace('\n,','').replace('\t','').strip())

116일,
총928시간


> table.view 태그로 회차별 정보 한 번에 뽑기

In [10]:
# 전체 회차
len(soup.select('a.tool_tip_btn'))

14

In [11]:
# 회차별 만족도 
count = len(soup.select('a.tool_tip_btn')) + 1
for i in soup.select('table.view')[2:]:
    if i.text.find('훈련진행중') != -1:
        count -= 1
        print(f'{count}회차 : 훈련진행중')
    else:
        for j in i.select('td'):
            tableview = j.text.replace('\n','').replace('\t','').strip()
            if re.findall('[.\d]+\s점',tableview) != []:
                count -= 1
                print(f"{count}회차 : " + re.findall('[.\d]+\s점',tableview).pop().replace(' 점',''))

14회차 : 훈련진행중
13회차 : 훈련진행중
12회차 : 훈련진행중
11회차 : 훈련진행중
10회차 : 75.2
9회차 : 82.4
8회차 : 90.84
7회차 : 90
6회차 : 91.58
5회차 : 84.54
4회차 : 89.6
3회차 : 89.56
2회차 : 80.74
1회차 : 95.56


In [12]:
# 회차별 모집인원
count = len(soup.select('a.tool_tip_btn')) + 1
for i in soup.select('table.view')[2:]:
    if i.text.find('훈련진행중') != -1:
        count -= 1
        print(f'{count}회차 : 훈련진행중')
    else:
        for j in i.select('td'):
            if re.findall('\d+명/',j.text) != []:
                count -= 1
                print(f"{count}회차 : " + re.findall('\d+명/',j.text).pop().replace('/',''))

14회차 : 훈련진행중
13회차 : 훈련진행중
12회차 : 훈련진행중
11회차 : 훈련진행중
10회차 : 25명
9회차 : 25명
8회차 : 25명
7회차 : 25명
6회차 : 24명
5회차 : 25명
4회차 : 25명
3회차 : 25명
2회차 : 28명
1회차 : 28명


> 망한 코드

In [13]:
# ?????????????????????
# [(\d]+/ : 취업인원
# /\d+ : 수료인원
count = len(soup.select('a.tool_tip_btn')) + 1
break_count = 2 
comple = ''
entire = ''
for i in soup.select('table.view'):
    if i.text.find('훈련진행중') != -1:
        count -= 1
        print(f'{count}회차 : 훈련진행중')
    else:
        for j in i.select('td'):
            break_count += 1
            if break_count % 3 == 0:
                if re.findall('\(/\)',j.text) != []:
                    count -= 1
                    print(f'{count}회차 : '+ re.findall('\(/\)',j.text).pop().strip('(').strip(')'))
#                     print('-')
#                     print('-')
                elif re.findall('[(\d]+/', j.text) != [] and re.findall('\d+\)', j.text) != []:
                    count -= 1
                    print(f'{count}회차 : '+ re.findall('[(\d]+/', j.text).pop().replace('(','') + re.findall('\d+\)', j.text).pop().replace(')',''))
#                     print(re.findall('[(\d]+/', j.text).pop().replace('(','')[:re.findall('[(\d]+/', j.text).pop().replace('(','').find('/')])
#                     print(re.findall('\d+\)', j.text).pop().replace(')','')[re.findall('\d+\)', j.text).pop().replace(')','').find('/')+1:])
#                     if re.findall('[(\d]+/', j.text) != []: # 훈련시작일 
#                         count -= 1
#                         comple = (re.findall('[(\d]+/', j.text).pop()) 
#                     if re.findall('\d+\)', j.text) != []:
#                         count -= 1
#                         entire = (re.findall('\d+\)', j.text).pop())
#                     print(f'{count}회차 : ',comple, entire)
            else:
                pass

14회차 : 훈련진행중
13회차 : 훈련진행중
12회차 : 훈련진행중
11회차 : 훈련진행중
10회차 : /
9회차 : /
8회차 : /23
7회차 : 20/23
6회차 : 1/19
5회차 : 15/18
4회차 : 14/25
3회차 : 2/23
2회차 : 16/21
1회차 : 21/26


> 다시 짠 코드

In [23]:
# 취업률 (고용보험가입 3개월) 취업인원/수료인원
empl_rate_3 = [] # 회차별 취업률(3개월)
empl_3 = [] # 회차별 취업인원 (3개월)
comp_3 = [] # 회차별 수료인원 (3개월)
series_count = len(soup.select('a.tool_tip_btn'))
# count = len(soup.select('a.tool_tip_btn')) + 1
cnt = 0
for i in soup.select('table.view'):
    if i.text.find('취업률 고용보험가입 3개월') != -1:
        for j in i.select('b'):
            cnt += 1
            if cnt % 3 == 1:
                rate_empl = j.text.replace('\n','').replace('\t','')
                empl_rate_3.append(rate_empl[:rate_empl.find("%")])
                empl_3.append(rate_empl[rate_empl.find("%")+2:rate_empl.find("/")])
                comp_3.append(rate_empl[rate_empl.find("/")+1:-1])
#                 print(f'취업률 : {rate_empl[:rate_empl.find("%")]}')
#                 print(f'취업인원 : {rate_empl[rate_empl.find("%")+2:rate_empl.find("/")]}')
#                 print(f'수료인원 : {rate_empl[rate_empl.find("/")+1:-1]}')

while len(empl_rate_3) != series_count:
    empl_rate_3.insert(0,'훈련진행중')
while len(empl_3) != series_count:
    empl_3.insert(0,'훈련진행중')
while len(comp_3) != series_count:
    comp_3.insert(0,'훈련진행중')
print(empl_rate_3)
print(empl_3)
print(comp_3)
len(empl_rate_3), len(empl_3), len(comp_3)

['훈련진행중', '훈련진행중', '훈련진행중', '훈련진행중', '', '', '60.9', '87', '63.2', '0', '56', '73.9', '57.1', '80.8']
['훈련진행중', '훈련진행중', '훈련진행중', '훈련진행중', '', '', '14', '20', '12', '0', '14', '17', '12', '21']
['훈련진행중', '훈련진행중', '훈련진행중', '훈련진행중', '', '', '23', '23', '19', '18', '25', '23', '21', '26']


(14, 14, 14)

In [24]:
# 취업률 (고용보험가입 6개월) 취업인원/수료인원
empl_rate_6 = [] # 회차별 취업률(6개월)
empl_6 = [] # 회차별 취업인원 (6개월)
comp_6 = [] # 회차별 수료인원 (6개월)
series_count = len(soup.select('a.tool_tip_btn'))
# count = len(soup.select('a.tool_tip_btn')) + 1
cnt = 0
for i in soup.select('table.view'):
    if i.text.find('취업률 고용보험가입 6개월') != -1:
        for j in i.select('b'):
            cnt += 1
            if cnt % 3 == 2:
                rate_empl = j.text.replace('\n','').replace('\t','')
                empl_rate_6.append(rate_empl[:rate_empl.find("%")])
                empl_6.append(rate_empl[rate_empl.find("%")+2:rate_empl.find("/")])
                comp_6.append(rate_empl[rate_empl.find("/")+1:-1])
#                 print(f'취업률 : {rate_empl[:rate_empl.find("%")]}')
#                 print(f'취업인원 : {rate_empl[rate_empl.find("%")+2:rate_empl.find("/")]}')
#                 print(f'수료인원 : {rate_empl[rate_empl.find("/")+1:-1]}')

while len(empl_rate_6) != series_count:
    empl_rate_6.insert(0,'훈련진행중')
while len(empl_6) != series_count:
    empl_6.insert(0,'훈련진행중')
while len(comp_6) != series_count:
    comp_6.insert(0,'훈련진행중')
print(empl_rate_6)
print(empl_6)
print(comp_6)
len(empl_rate_6), len(empl_6), len(comp_6)

['훈련진행중', '훈련진행중', '훈련진행중', '훈련진행중', '', '', '', '', '84.2', '83.3', '72', '82.6', '76.2', '92.3']
['훈련진행중', '훈련진행중', '훈련진행중', '훈련진행중', '', '', '', '', '16', '15', '18', '19', '16', '24']
['훈련진행중', '훈련진행중', '훈련진행중', '훈련진행중', '', '', '23', '23', '19', '18', '25', '23', '21', '26']


(14, 14, 14)

In [16]:
count = len(soup.select('a.tool_tip_btn')) + 1
for series in soup.select('span.con')[17:]:
    if series.text.find('~') != -1:
        count -= 1
        print(f'{count}회차 : ')
        print(series.text.replace('\n','').replace('\t','').strip()[:series.text.replace('\n','').replace('\t','').strip().find('~')])
        print(series.text.replace('\n','').replace('\t','').strip()[series.text.replace('\n','').replace('\t','').strip().find('~')+1:])

14회차 : 
2022-05-02 
2022-10-18
13회차 : 
2022-03-14 
2022-08-24
12회차 : 
2022-02-15 
2022-08-04
11회차 : 
2022-02-15 
2022-08-04
10회차 : 
2022-01-10 
2022-06-23
9회차 : 
2021-12-01 
2022-05-17
8회차 : 
2021-11-15 
2022-04-27
7회차 : 
2021-09-15 
2022-02-28
6회차 : 
2021-08-25 
2022-01-28
5회차 : 
2021-04-01 
2021-08-31
4회차 : 
2021-03-17 
2021-08-19
3회차 : 
2021-03-15 
2021-08-13
2회차 : 
2020-12-07 
2021-05-21
1회차 : 
2020-11-02 
2021-04-12


In [17]:
# 수강확정인원, 수강신청인원
for a in soup.select('table.view'):
    for b in a.select('td'):
        if b.text.find('수강확정인원') != -1:
            print(re.findall('\d+',b.text))

['25', '25']
['25', '25']
['25', '25']
['25', '25']
['25', '25']
['25', '25']
['25', '25']
['25', '25']
['24', '24']
['25', '25']
['25', '25']
['25', '25']
['28', '28']
['28', '28']


### 한 페이지에서 얻을 수 있는 모든 정보 얻음
---
- 회차별 만족도

- 회차별 모집인원

- 회차별 취업인원/수료인원

- 훈련시작일 ~훈련종료일

- 수강확정인원, 수강신청인원

### 페이지 내 링크 통해 찾을 수 있는 데이터
---
> N회차 보러가기 링크 들어가서 안에서 찾을 수 있는 것을 찾기
- 기관명
- 과정명
- 훈련비
- 훈련기간

1. N회차 링크 따기 ok

> column 확인용
1. 기관명 ok
2. 과정명 ok
3. 훈련비(원) ok
4. 훈련시간 ok
5. NCS 직무분류 ok
6. 수강생 평균 연령대(세) ok
7. 수강생 평균 만족도 ok
8. 회차 ok
9. 훈련기간 ok
10. 훈련시작일 ok
11. 훈련종료일 ok
12. 모집인원 ok
13. 수강확정인원 ok
14. 수강신청인원 ok
15. 회차별 만족도 ok
16. 취업률 고용보험가입 3개월 ok
17. 취업인원 ok
18. 수료인원 ok

## 1차 페이지 탐색 크롤링 시작

In [18]:
# 회차별 만족도
print('-'*15,'회차별 만족도','-'*15)
count = len(soup.select('a.tool_tip_btn')) + 1
for i in soup.select('table.view')[2:]:
    if i.text.find('훈련진행중') != -1:
        count -= 1
        print(f'{count}회차 : 훈련진행중')
    else:
        for j in i.select('td'):
            tableview = j.text.replace('\n','').replace('\t','').strip()
            if re.findall('[.\d]+\s점',tableview) != []:
                count -= 1
                print(f"{count}회차 : " + re.findall('[.\d]+\s점',tableview).pop().replace(' 점',''))

# 회차별 훈련기간
print('-'*15,'회차별 훈련기간','-'*15)
count = len(soup.select('a.tool_tip_btn')) + 1
for series in soup.select('span.con')[17:]:
    if series.text.find('~') != -1:
        count -= 1
        print(f'{count}회차 : ' + series.text.replace('\n','').replace('\t','').strip())
        
# 회차별 모집인원
print('-'*15,'회차별 모집인원','-'*15)
count = len(soup.select('a.tool_tip_btn')) + 1
for i in soup.select('table.view')[2:]:
    if i.text.find('훈련진행중') != -1:
        count -= 1
        print(f'{count}회차 : 훈련진행중')
    else:
        for j in i.select('td'):
            if re.findall('\d+명/',j.text) != []:
                count -= 1
                print(f"{count}회차 : " + re.findall('\d+명/',j.text).pop().replace('/',''))


# 회차별 수료, 취업 인원
print('-'*15,'회차별 [수료, 취업]인원','-'*15)
# [(\d]+/ : 취업인원
# /\d+ : 수료인원
count = len(soup.select('a.tool_tip_btn')) + 1
break_count = 2 
comple = ''
entire = ''
for i in soup.select('table.view'):
    if i.text.find('훈련진행중') != -1:
        count -= 1
        print(f'{count}회차 : 훈련진행중')
    else:
        for j in i.select('td'):
            break_count += 1
            if break_count % 3 == 0:
                if re.findall('\(/\)',j.text) != []:
                    count -= 1
                    print(f'{count}회차 : ' +re.findall('\(/\)',j.text).pop())
                elif re.findall('[(\d]+/', j.text) != [] and re.findall('\d+\)', j.text) != []:
                    count -= 1
                    print(f'{count}회차 : '+(re.findall('[(\d]+/', j.text).pop().replace('(','')) + re.findall('\d+\)', j.text).pop().replace(')',''))
            else:
                pass
            
# 회차별 수강확정인원, 수강신청인원
print('-'*15,'회차별 수강 [확정,신청] 인원','-'*15)
for a in soup.select('table.view'):
    for b in a.select('td'):
        if b.text.find('수강확정인원') != -1:
            print(re.findall('\d+',b.text))
for index, patterns in enumerate(soup.select('a.tool_tip_btn')):
    series_url = f'https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20200000286660&tracseTme={index+1}&crseTracseSe=C0061&trainstCstmrId=500020018488#undefined'
    resp_series = requests.get(series_url)
    soup_series = BeautifulSoup(resp_series.text, "html.parser")
    
    print(f'----------------{index+1} 회차 ---------------')
    
    # 차단 방지 시간 딜레이
    seed = np.random.randint(100)
    np.random.seed(seed)
    a = np.random.randint(5)
    time.sleep(a)
    
    # 기관명
    for organ in soup_series.select('p.add'):
        print(organ.text.strip())
#     data.append(organ.text.strip())
    
    # 과정명
    for course in soup_series.select('h4.tit'):
        print(course.text.replace('\n','').replace('\t',''))
#     data.append(course.text.replace('\n','').replace('\t',''))
    
    # 훈련비
    for price in soup_series.select('p.price'):
        print(price.text.strip().replace('\n','')[:price.text.find('원')].strip().replace('\t',''))
#     data.append(price.text.strip().replace('\n','')[:price.text.find('원')].strip().replace('\t',''))

    # 훈련시간
    for i in soup_series.select('span.con'):
        if i.text.find('일') != -1:
            print(i.text.replace('\n,','').replace('\t','').strip())
#             data.append(i.text.replace('\n,','').replace('\t','').strip())
    # NCS 자격분류
    for NCS in soup_series.select('span.con'):
        if re.findall('\d{6,}',NCS.text) != []:
            print(re.findall('.+\d{6,}.',NCS.text).pop().replace('\t',''))


--------------- 회차별 만족도 ---------------
14회차 : 훈련진행중
13회차 : 훈련진행중
12회차 : 훈련진행중
11회차 : 훈련진행중
10회차 : 75.2
9회차 : 82.4
8회차 : 90.84
7회차 : 90
6회차 : 91.58
5회차 : 84.54
4회차 : 89.6
3회차 : 89.56
2회차 : 80.74
1회차 : 95.56
--------------- 회차별 훈련기간 ---------------
14회차 : 2022-05-02 ~2022-10-18
13회차 : 2022-03-14 ~2022-08-24
12회차 : 2022-02-15 ~2022-08-04
11회차 : 2022-02-15 ~2022-08-04
10회차 : 2022-01-10 ~2022-06-23
9회차 : 2021-12-01 ~2022-05-17
8회차 : 2021-11-15 ~2022-04-27
7회차 : 2021-09-15 ~2022-02-28
6회차 : 2021-08-25 ~2022-01-28
5회차 : 2021-04-01 ~2021-08-31
4회차 : 2021-03-17 ~2021-08-19
3회차 : 2021-03-15 ~2021-08-13
2회차 : 2020-12-07 ~2021-05-21
1회차 : 2020-11-02 ~2021-04-12
--------------- 회차별 모집인원 ---------------
14회차 : 훈련진행중
13회차 : 훈련진행중
12회차 : 훈련진행중
11회차 : 훈련진행중
10회차 : 25명
9회차 : 25명
8회차 : 25명
7회차 : 25명
6회차 : 24명
5회차 : 25명
4회차 : 25명
3회차 : 25명
2회차 : 28명
1회차 : 28명
--------------- 회차별 [수료, 취업]인원 ---------------
14회차 : 훈련진행중
13회차 : 훈련진행중
12회차 : 훈련진행중
11회차 : 훈련진행중
10회차 : (/)
9회차 : (/)
8회차 : /23
7회차 : 20/23
6회차 :

## 1차 DateFrame 만들기

### Define column

> 기존에서 순서 바꾸고, 중복 제거

순서이렇게 하자

> 기관명, 과정명, 회차, 훈련비, 훈련시작일, 훈련종료일, 훈련시간, NCS 직무분류, 모집인원, 수강확정인원, 수강신청인원, 수강생 평균 연령대(세), 수강생 만족도, 취업률 고용보험가입 3개월, 취업인원(3개월), 수료인원, 취업률 고용보험가입 6개월, 취업인원(6개월)

> column size : 18

- 훈련시간, 회차별 만족도 -> 중복으로 뺌 

In [19]:
col = ['기관명', '과정명', '회차', '훈련비', '훈련시작일', '훈련종료일', 
       '훈련시간', 'NCS 직무분류', '모집인원', '수강확정인원', '수강신청인원', 
       '수강생 평균 연령대(세)', '수강생 만족도', '취업률 고용보험가입 3개월', 
       '취업인원(3개월)', '수료인원','취업률 고용보험가입 6개월', '취업인원(6개월)']
len(col)

18

## 데이터 프레임 열만 생성

In [20]:
df = pd.DataFrame(columns = col)
df

,기관명,과정명,회차,훈련비,훈련시작일,훈련종료일,훈련시간,NCS 직무분류,모집인원,수강확정인원,수강신청인원,수강생 평균 연령대(세),수강생 만족도,취업률 고용보험가입 3개월,취업인원(3개월),수료인원,취업률 고용보험가입 6개월,취업인원(6개월)


## 데이터 추가

> 1. concat : 추가할 대상이 데이터 프레임

> 2. append : 추가할 대상이 딕셔너리 -> 컬럼 이름 : key, 데이터 : value

> 3. loc : 추가할 대상이 리스트 -> 이때 loc에 들어가는 인덱스는 맨 마지막 인덱스로 설정.

일단 뭐든 시도

회차별로 한 번에 뽑을 수 있는 데이터는 따로 모을 예정

### dictionary로 만들어서 DataFrame에 append 해야 겠다.
---

> 각 컬럼 종류별로 리스트로 넣고 join 함수로 str로 바꿔서 dictionary에 넣고 미리 만든 df랑 append로 합치기

> frame.append 메소드가 나중 pandas에서는 사라질거라는 경고문.

> pandas.concat을 쓰라는 권고

> 나중을 위해 loc로 변경

## 2차 크롤링
---
list로 데이터 담기 시작

In [70]:
# 베이스 url
url = 'https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20200000286660&tracseTme=1&crseTracseSe=C0061&trainstCstmrId=500020018488#undefined'
resp = requests.get(url)
soup = BeautifulSoup(resp.text, 'html.parser')

# columns (중복 일부러 하나 둠)
orga = []# 기관명
cour = []# 과정명
seri = [str(s+1) for s in range(len(soup.select('a.tool_tip_btn')))] # 회차
seri.reverse()
pric = []# 훈련비
prev = [] # 회차별 훈련시작 +
fini = [] # 회차별 훈련종료 +
spen = []# 훈련시간
NCS_dist = []# NCS 직무 분류
recu = [] # 회차별 모집인원 +
conf = [] # 회차별 수강확정인원 +
appl = [] # 회차별 수강신청인원 +
mean_age = []# 수강생 평균 연령대
sati = [] # 회차별 만족도 +
empl_rate_3 = [] # 회차별 취업률(3개월) +
empl_3 = [] # 회차별 취업인원 (3개월) +
comp_3 = [] # 회차별 수료인원 (3개월) +
empl_rate_6 = [] # 회차별 취업률(6개월) +
empl_6 = [] # 회차별 취업인원 (6개월) +
comp_6 = [] # 회차별 수료인원 (6개월) +



# 회차별 만족도
print('-'*15,'회차별 만족도','-'*15)
count = len(soup.select('a.tool_tip_btn')) + 1
for i in soup.select('table.view')[2:]:
    if i.text.find('훈련진행중') != -1:
        count -= 1
        sati.append('훈련진행중')
        print(f'{count}회차 만족도 수집 완료')
    else:
        for j in i.select('td'):
            tableview = j.text.replace('\n','').replace('\t','').strip()
            if re.findall('[.\d]+\s점',tableview) != []:
                count -= 1
                sati.append(re.findall('[.\d]+\s점',tableview).pop().replace(' 점',''))
                print(f"{count}회차 만족도 수집 완료")
print('회차별 만족도 결과')
print(sati)
print(f'size : {len(sati)}')

# 회차별 훈련기간
print('-'*15,'회차별 훈련기간','-'*15)
count = len(soup.select('a.tool_tip_btn')) + 1
for series in soup.select('span.con')[17:]:
    if series.text.find('~') != -1:
        count -= 1
        print(f'{count}회차 훈련 시작일, 종료일 수집 완료')
        prev.append(series.text.replace('\n','').replace('\t','').strip()[:series.text.replace('\n','').replace('\t','').strip().find('~')])
        fini.append(series.text.replace('\n','').replace('\t','').strip()[series.text.replace('\n','').replace('\t','').strip().find('~')+1:])
#         print(series.text.replace('\n','').replace('\t','').strip()[:series.text.replace('\n','').replace('\t','').strip().find('~')])
#         print(series.text.replace('\n','').replace('\t','').strip()[series.text.replace('\n','').replace('\t','').strip().find('~')+1:])
print('회차별 시작일')
print(prev)
print(f'size : {len(prev)}')
print('회차별 종료일')
print(fini)
print(f'size : {len(fini)}')

# 회차별 모집인원
print('-'*15,'회차별 모집인원','-'*15)
count = len(soup.select('a.tool_tip_btn')) + 1
for i in soup.select('table.view')[2:]:
    if i.text.find('훈련진행중') != -1:
        count -= 1
        recu.append('훈련진행중')
        print(f'{count}회차 모집인원 수집 완료')
    else:
        for j in i.select('td'):
            if re.findall('\d+명/',j.text) != []:
                count -= 1
                recu.append(re.findall('\d+명/',j.text).pop().replace('명/',''))
                print(f"{count}회차 : 모집인원 수집 완료")
print('회차별 모집인원')
print(recu)
print(f'size : {len(recu)}')
                      
# 취업률 (고용보험가입 3개월) 취업인원/수료인원
series_count = len(soup.select('a.tool_tip_btn'))
# count = len(soup.select('a.tool_tip_btn')) + 1
cnt = 0
for i in soup.select('table.view'):
    if i.text.find('취업률 고용보험가입 3개월') != -1:
        for j in i.select('b'):
            cnt += 1
            if cnt % 3 == 1:
                rate_empl = j.text.replace('\n','').replace('\t','')
                empl_rate_3.append(rate_empl[:rate_empl.find("%")])
                empl_3.append(rate_empl[rate_empl.find("%")+2:rate_empl.find("/")])
                comp_3.append(rate_empl[rate_empl.find("/")+1:-1])
                print('취업률 수집 완료')
while len(empl_rate_3) != series_count:
    empl_rate_3.insert(0,'훈련진행중')
while len(empl_3) != series_count:
    empl_3.insert(0,'훈련진행중')
while len(comp_3) != series_count:
    comp_3.insert(0,'훈련진행중')
print(empl_rate_3)
print(f'size : {len(empl_rate_3)}')
print(empl_3)
print(f'size : {len(empl_3)}')
print(comp_3)
print(f'size : {len(comp_3)}')


# 취업률 (고용보험가입 6개월) 취업인원/수료인원
series_count = len(soup.select('a.tool_tip_btn'))
# count = len(soup.select('a.tool_tip_btn')) + 1
cnt = 0
for i in soup.select('table.view'):
    if i.text.find('취업률 고용보험가입 6개월') != -1:
        for j in i.select('b'):
            cnt += 1
            if cnt % 3 == 2:
                rate_empl = j.text.replace('\n','').replace('\t','')
                empl_rate_6.append(rate_empl[:rate_empl.find("%")])
                empl_6.append(rate_empl[rate_empl.find("%")+2:rate_empl.find("/")])
                comp_6.append(rate_empl[rate_empl.find("/")+1:-1])
                
while len(empl_rate_6) != series_count:
    empl_rate_6.insert(0,'훈련진행중')
while len(empl_6) != series_count:
    empl_6.insert(0,'훈련진행중')
while len(comp_6) != series_count:
    comp_6.insert(0,'훈련진행중')
print(empl_rate_6)
print(f'size : {len(empl_rate_6)}')
print(empl_6)
print(f'size : {len(empl_6)}')
print(comp_6)
print(f'size : {len(comp_6)}')

# 회차별 수강확정인원, 수강신청인원
print('-'*15,'회차별 수강 [확정,신청] 인원','-'*15)
for a in soup.select('table.view'):
    for b in a.select('td'):
        if b.text.find('수강확정인원') != -1:
#             print(re.findall('\d+',b.text))
            conf.append(re.findall('\d+',b.text)[0])
            appl.append(re.findall('\d+',b.text)[1])
print('수강확정인원')
print(conf)
print(f'size : {len(conf)}')
print('수강신청인원')
print(appl)
print(f'size : {len(appl)}')

count = len(soup.select('a.tool_tip_btn')) + 1
    
for patterns in soup.select('a.tool_tip_btn'):
    series_url = f'https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20200000286660&tracseTme={index+1}&crseTracseSe=C0061&trainstCstmrId=500020018488#undefined'
    resp_series = requests.get(series_url)
    soup_series = BeautifulSoup(resp_series.text, "html.parser")

    count -= 1
    print(f'---------------- {count} 회차 ---------------')
    
    # 차단 방지 시간 딜레이
    seed = np.random.randint(100)
    np.random.seed(seed)
    a = np.random.randint(5)
    time.sleep(a)
    
    # 기관명
    for organ in soup_series.select('p.add'):
        orga.append(organ.text.strip())

    # 과정명
    for course in soup_series.select('h4.tit'):
#         print(course.text.replace('\n','').replace('\t',''))
        cour.append(course.text.replace('\n','').replace('\t',''))
    
    # 훈련비
    pric.append(soup.select('p.price')[0].text[:soup.select('p.price')[0].text.find(' 원')].replace('\n', '').replace('\t', ''))
#     for price in soup_series.select('p.price'):
#         print(price.text.strip().replace('\n','')[:price.text.find('원')].strip().replace('\t',''))
#         pric.append(price.text.strip().replace('\n','')[:price.text.find('원')].strip().replace('\t',''))

    # 훈련시간
    for i in soup.select('span.con'):
        if i.text.find('일') != -1:
#             print(i.text.replace('\n','').replace('\t',''))
            spen.append(i.text.replace('\n,','').replace('\t','').strip().replace('\n',''))
    # NCS 자격분류
    for NCS in soup_series.select('span.con'):
        if re.findall('\d{6,}',NCS.text) != []:
#             print(re.findall('.+\d{6,}.',NCS.text).pop().replace('\t',''))
            NCS_dist.append(re.findall('.+\d{6,}.',NCS.text).pop().replace('\t',''))
    # 수강생 평균나이
    for g in soup.select('span.con'):
        if g.text.find('세') != -1:
#         print(re.findall("\d+\s세", g.text).pop().replace(' 세',''))
            mean_age.append(re.findall("\d+\s세", g.text).pop().replace(' 세',''))

print(orga)
print(f'size : {len(orga)}')
print(cour)
print(f'size : {len(cour)}')
print(pric)
print(f'size : {len(pric)}')
print(spen)
print(f'size : {len(spen)}')
print(NCS_dist)
print(f'size : {len(NCS_dist)}')

--------------- 회차별 만족도 ---------------
14회차 만족도 수집 완료
13회차 만족도 수집 완료
12회차 만족도 수집 완료
11회차 만족도 수집 완료
10회차 만족도 수집 완료
9회차 만족도 수집 완료
8회차 만족도 수집 완료
7회차 만족도 수집 완료
6회차 만족도 수집 완료
5회차 만족도 수집 완료
4회차 만족도 수집 완료
3회차 만족도 수집 완료
2회차 만족도 수집 완료
1회차 만족도 수집 완료
회차별 만족도 결과
['훈련진행중', '훈련진행중', '훈련진행중', '훈련진행중', '75.2', '82.4', '90.84', '90', '91.58', '84.54', '89.6', '89.56', '80.74', '95.56']
size : 14
--------------- 회차별 훈련기간 ---------------
14회차 훈련 시작일, 종료일 수집 완료
13회차 훈련 시작일, 종료일 수집 완료
12회차 훈련 시작일, 종료일 수집 완료
11회차 훈련 시작일, 종료일 수집 완료
10회차 훈련 시작일, 종료일 수집 완료
9회차 훈련 시작일, 종료일 수집 완료
8회차 훈련 시작일, 종료일 수집 완료
7회차 훈련 시작일, 종료일 수집 완료
6회차 훈련 시작일, 종료일 수집 완료
5회차 훈련 시작일, 종료일 수집 완료
4회차 훈련 시작일, 종료일 수집 완료
3회차 훈련 시작일, 종료일 수집 완료
2회차 훈련 시작일, 종료일 수집 완료
1회차 훈련 시작일, 종료일 수집 완료
회차별 시작일
['2022-05-02 ', '2022-03-14 ', '2022-02-15 ', '2022-02-15 ', '2022-01-10 ', '2021-12-01 ', '2021-11-15 ', '2021-09-15 ', '2021-08-25 ', '2021-04-01 ', '2021-03-17 ', '2021-03-15 ', '2020-12-07 ', '2020-11-02 ']
size : 14
회차별 종료일
['2022-10-18', '2022-08-24

### list -> str -> dictionary 로 변환

In [286]:
li_list =[orga,cour,seri,pric,prev,fini,spen,NCS_dist,recu,conf,appl,mean_age,sati,empl_rate_3,empl_3,comp_3,empl_rate_6,empl_6]
for li in li_list:
    print(li)

['스마트인재개발원', '스마트인재개발원', '스마트인재개발원', '스마트인재개발원', '스마트인재개발원', '스마트인재개발원', '스마트인재개발원', '스마트인재개발원', '스마트인재개발원', '스마트인재개발원', '스마트인재개발원', '스마트인재개발원', '스마트인재개발원', '스마트인재개발원']
['빅데이터 분석서비스 개발자과정(NCS)모집마감', '빅데이터 분석서비스 개발자과정(NCS)모집마감', '빅데이터 분석서비스 개발자과정(NCS)모집마감', '빅데이터 분석서비스 개발자과정(NCS)모집마감', '빅데이터 분석서비스 개발자과정(NCS)모집마감', '빅데이터 분석서비스 개발자과정(NCS)모집마감', '빅데이터 분석서비스 개발자과정(NCS)모집마감', '빅데이터 분석서비스 개발자과정(NCS)모집마감', '빅데이터 분석서비스 개발자과정(NCS)모집마감', '빅데이터 분석서비스 개발자과정(NCS)모집마감', '빅데이터 분석서비스 개발자과정(NCS)모집마감', '빅데이터 분석서비스 개발자과정(NCS)모집마감', '빅데이터 분석서비스 개발자과정(NCS)모집마감', '빅데이터 분석서비스 개발자과정(NCS)모집마감']
['14', '13', '12', '11', '10', '9', '8', '7', '6', '5', '4', '3', '2', '1']
['13,762,240', '13,762,240', '13,762,240', '13,762,240', '13,762,240', '13,762,240', '13,762,240', '13,762,240', '13,762,240', '13,762,240', '13,762,240', '13,762,240', '13,762,240', '13,762,240']
['2022-05-02 ', '2022-03-14 ', '2022-02-15 ', '2022-02-15 ', '2022-01-10 ', '2021-12-01 ', '2021-11-15 ', '2021-09-15 ', '2021-08-25 ', '2021-04-01 ', 

> list to str

### dictionary -> DataFrame 변환

In [287]:
# pd.DataFrame(columns = colu)
df = pd.DataFrame(columns = col)
df

,기관명,과정명,회차,훈련비,훈련시작일,훈련종료일,훈련시간,NCS 직무분류,모집인원,수강확정인원,수강신청인원,수강생 평균 연령대(세),수강생 만족도,취업률 고용보험가입 3개월,취업인원(3개월),수료인원,취업률 고용보험가입 6개월,취업인원(6개월)


### loc 사용

#### 두 리스트 동시에 사용하기 위해 for zip 사용

In [490]:
dic_test = {}
for li, cols in zip(li_list, df.columns.tolist()):
    dic_test[cols]= li
pd.DataFrame(dic_test)

pd.concat([df, pd.DataFrame(dic_test)], ignore_index=True)

,기관명,과정명,회차,훈련비,훈련시작일,훈련종료일,훈련시간,NCS 직무분류,모집인원,수강확정인원,수강신청인원,수강생 평균 연령대(세),수강생 만족도,취업률 고용보험가입 3개월,취업인원(3개월),수료인원,취업률 고용보험가입 6개월,취업인원(6개월)
0,스마트인재개발원,빅데이터 분석서비스 개발자과정(NCS)모집마감,4,"6,219,200",2022-11-24,2023-04-17,"100일,총800시간",응용SW엔지니어링(20010202),0,0,0,-,0,훈련진행중,훈련진행중,훈련진행중,훈련진행중,훈련진행중
1,스마트인재개발원,빅데이터 분석서비스 개발자과정(NCS)모집마감,3,"6,219,200",2022-10-27,2023-03-20,"100일,총800시간",응용SW엔지니어링(20010202),0,0,0,-,0,훈련진행중,훈련진행중,훈련진행중,훈련진행중,훈련진행중
2,스마트인재개발원,빅데이터 분석서비스 개발자과정(NCS)모집마감,2,"6,219,200",2022-06-17,2022-11-10,"100일,총800시간",응용SW엔지니어링(20010202),훈련진행중,12,12,-,훈련진행중,훈련진행중,훈련진행중,훈련진행중,훈련진행중,훈련진행중
3,스마트인재개발원,빅데이터 분석서비스 개발자과정(NCS)모집마감,1,"6,219,200",2021-07-09,2021-11-30,"100일,총800시간",응용SW엔지니어링(20010202),19,19,19,-,90.76,66.7,10,15,80,12
4,스마트인재개발원,빅데이터 분석서비스 개발자과정(NCS)모집마감,4,"6,219,200",2022-11-24,2023-04-17,"100일,총800시간",응용SW엔지니어링(20010202),0,0,0,-,0,훈련진행중,훈련진행중,훈련진행중,훈련진행중,훈련진행중
5,스마트인재개발원,빅데이터 분석서비스 개발자과정(NCS)모집마감,3,"6,219,200",2022-10-27,2023-03-20,"100일,총800시간",응용SW엔지니어링(20010202),0,0,0,-,0,훈련진행중,훈련진행중,훈련진행중,훈련진행중,훈련진행중
6,스마트인재개발원,빅데이터 분석서비스 개발자과정(NCS)모집마감,2,"6,219,200",2022-06-17,2022-11-10,"100일,총800시간",응용SW엔지니어링(20010202),훈련진행중,12,12,-,훈련진행중,훈련진행중,훈련진행중,훈련진행중,훈련진행중,훈련진행중
7,스마트인재개발원,빅데이터 분석서비스 개발자과정(NCS)모집마감,1,"6,219,200",2021-07-09,2021-11-30,"100일,총800시간",응용SW엔지니어링(20010202),19,19,19,-,90.76,66.7,10,15,80,12


In [468]:
var_list = ['orga','cour','seri','pric','prev','fini','spen','NCS_dist','recu','conf','appl','mean_age','sati','empl_rate_3','empl_3','comp_3','empl_rate_6','empl_6']

for li, cols in zip(li_list, df.columns.tolist()):
    df.loc[cols] = li
df

ValueError: cannot set a row with mismatched columns

## 엑셀로 저장
---

In [290]:
df.to_excel('HRDcrawling.xlsx', index=False)

# HRD crawling 2
---
모든 페이지 내 존재하는 주소 가져오기

In [291]:
kdt_url = 'https://www.hrd.go.kr/hrdp/ti/ptiao/PTIAO0100L.do'
kdt_resp = requests.get(kdt_url)
kdt_soup = BeautifulSoup(kdt_resp.text, "html.parser")

In [295]:
kdt_soup.select('a.pb2.next')[0].get('href')

'?pageIndex=1715'

## k-디지털 트레이닝 주소 
---
숨겨놔서 찾은게 놀라움 </br>
https://www.hrd.go.kr/hrdp/ti/ptiao/PTIAO0100L.do?pageId=6&bgrllnstYn=undefined&DgtlYn=undefined&_csrf=6b34cee0-bc6e-4e71-b844-74891e83d22e

> html 전부 뜯어보니 더 간단한 주소 찾음

https://www.hrd.go.kr/hrdp/ti/ptiao/PTIAO0100L.do?pageId=2&kDgtlYn=Y

## k-디지털 기초역량 훈련 주소
---
undefine 부분이 바뀌면서 url 패턴찾는거 막는듯 </br>
https://www.hrd.go.kr/hrdp/ti/ptiao/PTIAO0100L.do?pageId=2&bgrllnstYn=&kDgtlYn=Y&_csrf=6b34cee0-bc6e-4e71-b844-74891e83d22e

> 위와 마찬가지로 뜯어보니 간단한 주소 찾음

https://www.hrd.go.kr/hrdp/ti/ptiao/PTIAO0100L.do?pageId=2&kDgtlYn=Y

# HRD-net
---
HRD 홈페이지가 url 크롤링을 의도적으로 막은 것 같다.

대부분의 페이지를 동적페이지인것 마냥 url을 숨겨서 고정시켰다.

모든 html을 뜯어보니 url 비밀에 접근할 수 있었다.

> 동적페이지 인척하는 정적페이지

<img src='img/pageUrl.png'>

요 순서대로 url에 원하는 value를 넣으면 된다.

## k-디지털 트레이닝 크롤링

In [311]:
#'https://www.hrd.go.kr/hrdp/ti/ptiao/PTIAO0100L.do?pageId=6&bgrlInstYn=undefined&pageIndex=6&kDgtlYn=undefined&_csrf=6b34cee0-bc6e-4e71-b844-74801e83d22e'
pageIndex = 1 # 1페이지
kdt_url = f'https://www.hrd.go.kr/hrdp/ti/ptiao/PTIAO0100L.do?pageId=6&bgrlInstYn=undefined&pageIndex={pageIndex}&kDgtlYn=undefined&_csrf=6b34cee0-bc6e-4e71-b844-74801e83d22e'
kdt_resp = requests.get(kdt_url)
kdt_soup = BeautifulSoup(kdt_resp.text, "html.parser")

In [312]:
#마지막페이지 추출
end_page = re.findall('\d+',kdt_soup.select('a.pb2.next')[0].get('href')).pop()
end_page

'14'

In [318]:
# k-디지털 트레이닝 모든 페이지 주소
for pageIndex in range(int(end_page)):
    kdt_url = f'https://www.hrd.go.kr/hrdp/ti/ptiao/PTIAO0100L.do?pageId=6&bgrlInstYn=undefined&pageIndex={pageIndex+1}&kDgtlYn=undefined&_csrf=6b34cee0-bc6e-4e71-b844-74801e83d22e'
    print(kdt_url)

https://www.hrd.go.kr/hrdp/ti/ptiao/PTIAO0100L.do?pageId=6&bgrlInstYn=undefined&pageIndex=1&kDgtlYn=undefined&_csrf=6b34cee0-bc6e-4e71-b844-74801e83d22e
https://www.hrd.go.kr/hrdp/ti/ptiao/PTIAO0100L.do?pageId=6&bgrlInstYn=undefined&pageIndex=2&kDgtlYn=undefined&_csrf=6b34cee0-bc6e-4e71-b844-74801e83d22e
https://www.hrd.go.kr/hrdp/ti/ptiao/PTIAO0100L.do?pageId=6&bgrlInstYn=undefined&pageIndex=3&kDgtlYn=undefined&_csrf=6b34cee0-bc6e-4e71-b844-74801e83d22e
https://www.hrd.go.kr/hrdp/ti/ptiao/PTIAO0100L.do?pageId=6&bgrlInstYn=undefined&pageIndex=4&kDgtlYn=undefined&_csrf=6b34cee0-bc6e-4e71-b844-74801e83d22e
https://www.hrd.go.kr/hrdp/ti/ptiao/PTIAO0100L.do?pageId=6&bgrlInstYn=undefined&pageIndex=5&kDgtlYn=undefined&_csrf=6b34cee0-bc6e-4e71-b844-74801e83d22e
https://www.hrd.go.kr/hrdp/ti/ptiao/PTIAO0100L.do?pageId=6&bgrlInstYn=undefined&pageIndex=6&kDgtlYn=undefined&_csrf=6b34cee0-bc6e-4e71-b844-74801e83d22e
https://www.hrd.go.kr/hrdp/ti/ptiao/PTIAO0100L.do?pageId=6&bgrlInstYn=undefined&pa

In [345]:
for anc in kdt_soup.select('div.title'):
    for a in anc.select('a'):
        pattern = re.findall("'\w+\d*'",a.get('onclick'))
        print(pattern)

["'AIG20210000313652'", "'6'", "'500020059960'", "'C0061'", "'PC'"]
["'AIG20210000313685'", "'10'", "'500020013370'", "'C0061'", "'PC'"]
["'AIG20200000286570'", "'6'", "'500020042738'", "'C0061'", "'PC'"]
["'AIG20200000313547'", "'21'", "'500020039927'", "'C0061'", "'PC'"]
["'AIG20220000366834'", "'1'", "'500020013014'", "'C0061'", "'PC'"]
["'AIG20220000366893'", "'2'", "'500020028562'", "'C0061'", "'PC'"]
["'AIG20220000366743'", "'1'", "'500020039927'", "'C0061'", "'PC'"]
["'AIG20220000409455'", "'3'", "'500020044796'", "'C0061'", "'PC'"]
["'AIG20220000366776'", "'1'", "'500020040446'", "'C0061'", "'PC'"]
["'AIG20220000366733'", "'1'", "'500020039773'", "'C0061'", "'PC'"]


In [380]:
# https://www.hrd.go.kr/hrdp/co/pcoco/PCOCO0100P.do?tracseId=AIG20220000366733&tracseTme=1&trainstCstmrId=500020039773&crseTracseSe=C0061

#https://www.hrd.go.kr/hrdp/co/pcoco/PCOCO0100P.do?tracseId={AIG20220000366733}&tracseTme={1}&trainstCstmrId={500020039773}&crseTracseSe={C0061}

count_url = 0

# k-디지털 트레이닝 모든 페이지 주소
for pageIndex in range(int(end_page)):
    kdt_url = f'https://www.hrd.go.kr/hrdp/ti/ptiao/PTIAO0100L.do?pageId=6&bgrlInstYn=undefined&pageIndex={pageIndex+1}&kDgtlYn=undefined&_csrf=6b34cee0-bc6e-4e71-b844-74801e83d22e'
    kdt_resp = requests.get(kdt_url)
    kdt_soup = BeautifulSoup(kdt_resp.text, "html.parser")
    
    # 차단 방지 시간 딜레이
    seed = np.random.randint(100)
    np.random.seed(seed)
    a = np.random.randint(5)
    time.sleep(a)
    
    for anc in kdt_soup.select('div.title'):
        for a in anc.select('a'):
            pattern = []
            for p in re.findall("'\w+\d*'",a.get('onclick')):
                pattern.append(p.strip("'"))
            url = f"https://www.hrd.go.kr/hrdp/co/pcoco/PCOCO0100P.do?tracseId={pattern[0]}&tracseTme={pattern[1]}&trainstCstmrId={pattern[2]}&crseTracseSe={pattern[3]}"
            count_url += 1
            print(f' --------------- url 크롤링 {count_url} ---------------')
            print(url)

 --------------- url 크롤링 1 ---------------
https://www.hrd.go.kr/hrdp/co/pcoco/PCOCO0100P.do?tracseId=AIG20210000313652&tracseTme=6&trainstCstmrId=500020059960&crseTracseSe=C0061
 --------------- url 크롤링 2 ---------------
https://www.hrd.go.kr/hrdp/co/pcoco/PCOCO0100P.do?tracseId=AIG20210000313685&tracseTme=10&trainstCstmrId=500020013370&crseTracseSe=C0061
 --------------- url 크롤링 3 ---------------
https://www.hrd.go.kr/hrdp/co/pcoco/PCOCO0100P.do?tracseId=AIG20200000286570&tracseTme=6&trainstCstmrId=500020042738&crseTracseSe=C0061
 --------------- url 크롤링 4 ---------------
https://www.hrd.go.kr/hrdp/co/pcoco/PCOCO0100P.do?tracseId=AIG20200000313547&tracseTme=21&trainstCstmrId=500020039927&crseTracseSe=C0061
 --------------- url 크롤링 5 ---------------
https://www.hrd.go.kr/hrdp/co/pcoco/PCOCO0100P.do?tracseId=AIG20220000366834&tracseTme=1&trainstCstmrId=500020013014&crseTracseSe=C0061
 --------------- url 크롤링 6 ---------------
https://www.hrd.go.kr/hrdp/co/pcoco/PCOCO0100P.do?tracseId=A

 --------------- url 크롤링 51 ---------------
https://www.hrd.go.kr/hrdp/co/pcoco/PCOCO0100P.do?tracseId=AIG20210000328823&tracseTme=2&trainstCstmrId=500020034331&crseTracseSe=C0061
 --------------- url 크롤링 52 ---------------
https://www.hrd.go.kr/hrdp/co/pcoco/PCOCO0100P.do?tracseId=AIG20210000328590&tracseTme=5&trainstCstmrId=500020046871&crseTracseSe=C0061
 --------------- url 크롤링 53 ---------------
https://www.hrd.go.kr/hrdp/co/pcoco/PCOCO0100P.do?tracseId=AIG20220000366913&tracseTme=2&trainstCstmrId=500020027870&crseTracseSe=C0061
 --------------- url 크롤링 54 ---------------
https://www.hrd.go.kr/hrdp/co/pcoco/PCOCO0100P.do?tracseId=AIG20220000366708&tracseTme=1&trainstCstmrId=500036075789&crseTracseSe=C0061
 --------------- url 크롤링 55 ---------------
https://www.hrd.go.kr/hrdp/co/pcoco/PCOCO0100P.do?tracseId=AIG20220000366843&tracseTme=2&trainstCstmrId=500034091717&crseTracseSe=C0061
 --------------- url 크롤링 56 ---------------
https://www.hrd.go.kr/hrdp/co/pcoco/PCOCO0100P.do?tracse

 --------------- url 크롤링 101 ---------------
https://www.hrd.go.kr/hrdp/co/pcoco/PCOCO0100P.do?tracseId=AIG20220000366763&tracseTme=1&trainstCstmrId=500020024317&crseTracseSe=C0061
 --------------- url 크롤링 102 ---------------
https://www.hrd.go.kr/hrdp/co/pcoco/PCOCO0100P.do?tracseId=AIG20200000286611&tracseTme=10&trainstCstmrId=500020049185&crseTracseSe=C0061
 --------------- url 크롤링 103 ---------------
https://www.hrd.go.kr/hrdp/co/pcoco/PCOCO0100P.do?tracseId=AIG20220000366818&tracseTme=1&trainstCstmrId=500020045827&crseTracseSe=C0061
 --------------- url 크롤링 104 ---------------
https://www.hrd.go.kr/hrdp/co/pcoco/PCOCO0100P.do?tracseId=AIG20210000313688&tracseTme=3&trainstCstmrId=500033945638&crseTracseSe=C0061
 --------------- url 크롤링 105 ---------------
https://www.hrd.go.kr/hrdp/co/pcoco/PCOCO0100P.do?tracseId=AIG20210000314444&tracseTme=3&trainstCstmrId=500020049185&crseTracseSe=C0061
 --------------- url 크롤링 106 ---------------
https://www.hrd.go.kr/hrdp/co/pcoco/PCOCO0100P.do

In [ ]:
# https://www.hrd.go.kr/hrdp/co/pcoco/PCOCO0100P.do?tracseId=AIG20220000366733&tracseTme=1&trainstCstmrId=500020039773&crseTracseSe=C0061

#https://www.hrd.go.kr/hrdp/co/pcoco/PCOCO0100P.do?tracseId={AIG20220000366733}&tracseTme={1}&trainstCstmrId={500020039773}&crseTracseSe={C0061}

count_url = 0

# k-디지털 트레이닝 모든 페이지 주소
for pageIndex in range(int(end_page)):
    kdt_url = f'https://www.hrd.go.kr/hrdp/ti/ptiao/PTIAO0100L.do?pageId=6&bgrlInstYn=undefined&pageIndex={pageIndex+1}&kDgtlYn=undefined&_csrf=6b34cee0-bc6e-4e71-b844-74801e83d22e'
    kdt_resp = requests.get(kdt_url)
    kdt_soup = BeautifulSoup(kdt_resp.text, "html.parser")
    
    # 차단 방지 시간 딜레이
    seed = np.random.randint(100)
    np.random.seed(seed)
    a = np.random.randint(5)
    time.sleep(a)
    
    for anc in kdt_soup.select('div.title'):
        for a in anc.select('a'):
            pattern = []
            for index, p in enumerate(re.findall("'\w+\d*'",a.get('onclick'))):
                pattern.append(p.strip("'"))
            url = f"https://www.hrd.go.kr/hrdp/co/pcoco/PCOCO0100P.do?tracseId={pattern[0]}&tracseTme={pattern[1]}&trainstCstmrId={pattern[2]}&crseTracseSe={pattern[3]}"
            count_url += 1
            print(f' --------------- url 크롤링 {count_url} 번째 아이템 찾는 중  ---------------')
            

In [496]:
# https://www.hrd.go.kr/hrdp/co/pcoco/PCOCO0100P.do?tracseId=AIG20220000366733&tracseTme=1&trainstCstmrId=500020039773&crseTracseSe=C0061

#https://www.hrd.go.kr/hrdp/co/pcoco/PCOCO0100P.do?tracseId={AIG20220000366733}&tracseTme={1}&trainstCstmrId={500020039773}&crseTracseSe={C0061}

# 훈련정보 바로 나오는 링크
# 'https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20220000366816&tracseTme=2&trainstCstmrId=500020028586&crseTracseSe=C0061https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20220000366816&pageId='
count_url = 0

col = ['기관명', '과정명', '회차', '훈련비', '훈련시작일', '훈련종료일', 
       '훈련시간', 'NCS 직무분류', '모집인원', '수강확정인원', '수강신청인원', 
       '수강생 평균 연령대(세)', '수강생 만족도', '취업률 고용보험가입 3개월', 
       '취업인원(3개월)', '수료인원','취업률 고용보험가입 6개월', '취업인원(6개월)']

df = pd.DataFrame(columns = col)

# k-디지털 트레이닝 모든 페이지 주소
for pageIndex in range(int(end_page)):
    kdt_url = f'https://www.hrd.go.kr/hrdp/ti/ptiao/PTIAO0100L.do?pageId=6&bgrlInstYn=undefined&pageIndex={pageIndex+1}&kDgtlYn=undefined&_csrf=6b34cee0-bc6e-4e71-b844-74801e83d22e'
    print(f'{pageIndex+1} 크롤링 ...')
    kdt_resp = requests.get(kdt_url)
    kdt_soup = BeautifulSoup(kdt_resp.text, "html.parser")
    
    # 차단 방지 시간 딜레이
    seed = np.random.randint(100)
    np.random.seed(seed)
    a = np.random.randint(5)
    time.sleep(a)
    
    for anc in kdt_soup.select('div.title'):
        for a in anc.select('a'):
            pattern = []
            for index, p in enumerate(re.findall("'\w+\d*'",a.get('onclick'))):
                pattern.append(p.strip("'"))
            url = f'https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId={pattern[0]}&tracseTme={pattern[1]}&trainstCstmrId={pattern[2]}&crseTracseSe={pattern[3]}https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId={pattern[0]}&pageId='
#             url = f"https://www.hrd.go.kr/hrdp/co/pcoco/PCOCO0100P.do?tracseId={pattern[0]}&tracseTme={pattern[1]}&trainstCstmrId={pattern[2]}&crseTracseSe={pattern[3]}"
            count_url += 1
            print(f'{count_url} 번째 아이템 찾는 중...')
            print(f'링크 : {url}')
            
    # 베이스 url
#     url = 'https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20200000286660&tracseTme=1&crseTracseSe=C0061&trainstCstmrId=500020018488#undefined'
            resp = requests.get(url)
            soup = BeautifulSoup(resp.text, 'html.parser')

            # columns (중복 일부러 하나 둠)
            orga = []# 기관명
            cour = []# 과정명
            seri = [str(s+1) for s in range(len(soup.select('a.tool_tip_btn')))] # 회차
            seri.reverse()
            pric = []# 훈련비
            prev = [] # 회차별 훈련시작 +
            fini = [] # 회차별 훈련종료 +
            spen = []# 훈련시간
            NCS_dist = []# NCS 직무 분류
            recu = [] # 회차별 모집인원 +
            conf = [] # 회차별 수강확정인원 +
            appl = [] # 회차별 수강신청인원 +
            mean_age = []# 수강생 평균 연령대
            sati = [] # 회차별 만족도 +
            empl_rate_3 = [] # 회차별 취업률(3개월) +
            empl_3 = [] # 회차별 취업인원 (3개월) +
            comp_3 = [] # 회차별 수료인원 (3개월) +
            empl_rate_6 = [] # 회차별 취업률(6개월) +
            empl_6 = [] # 회차별 취업인원 (6개월) +
            comp_6 = [] # 회차별 수료인원 (6개월) +



            # 회차별 만족도
            print('-'*15,'회차별 만족도','-'*15)
            count = len(soup.select('a.tool_tip_btn')) + 1
            for i in soup.select('table.view')[2:]:
                if i.text.find('훈련진행중') != -1:
                    count -= 1
                    sati.append('훈련진행중')
                    print(f'{count}회차 만족도 수집중...')
                else:
                    for j in i.select('td'):
                        tableview = j.text.replace('\n','').replace('\t','').strip()
                        if re.findall('[.\d]+\s점',tableview) != []:
                            count -= 1
                            sati.append(re.findall('[.\d]+\s점',tableview).pop().replace(' 점',''))
                            print(f"{count}회차 만족도 수집중...")


            # 회차별 훈련기간
            print('-'*15,'회차별 훈련기간','-'*15)
            count = len(soup.select('a.tool_tip_btn')) + 1
            for series in soup.select('span.con')[17:]:
                if series.text.find('~') != -1:
                    count -= 1
                    print(f'{count}회차 훈련 시작일, 종료일 수집중...')
                    prev.append(series.text.replace('\n','').replace('\t','').strip()[:series.text.replace('\n','').replace('\t','').strip().find('~')])
                    fini.append(series.text.replace('\n','').replace('\t','').strip()[series.text.replace('\n','').replace('\t','').strip().find('~')+1:])
            #         print(series.text.replace('\n','').replace('\t','').strip()[:series.text.replace('\n','').replace('\t','').strip().find('~')])
            #         print(series.text.replace('\n','').replace('\t','').strip()[series.text.replace('\n','').replace('\t','').strip().find('~')+1:])


            # 회차별 모집인원
            print('-'*15,'회차별 모집인원','-'*15)
            count = len(soup.select('a.tool_tip_btn')) + 1
            for i in soup.select('table.view')[2:]:
                if i.text.find('훈련진행중') != -1:
                    count -= 1
                    recu.append('훈련진행중')
                    print(f'{count}회차 모집인원 수집중...')
                else:
                    for j in i.select('td'):
                        if re.findall('\d+명/',j.text) != []:
                            count -= 1
                            recu.append(re.findall('\d+명/',j.text).pop().replace('명/',''))
                            print(f"{count}회차 : 모집인원 수집중...")

            # 취업률 (고용보험가입 3개월) 취업인원/수료인원
            series_count = len(soup.select('a.tool_tip_btn'))
            # count = len(soup.select('a.tool_tip_btn')) + 1
            cnt = 0
            for i in soup.select('table.view'):
                if i.text.find('취업률 고용보험가입 3개월') != -1:
                    for j in i.select('b'):
                        cnt += 1
                        if cnt % 3 == 1:
                            rate_empl = j.text.replace('\n','').replace('\t','')
                            empl_rate_3.append(rate_empl[:rate_empl.find("%")])
                            empl_3.append(rate_empl[rate_empl.find("%")+2:rate_empl.find("/")])
                            comp_3.append(rate_empl[rate_empl.find("/")+1:-1])
                            print('취업률 수집 완중...')
            while len(empl_rate_3) != series_count:
                empl_rate_3.insert(0,'훈련진행중')
            while len(empl_3) != series_count:
                empl_3.insert(0,'훈련진행중')
            while len(comp_3) != series_count:
                comp_3.insert(0,'훈련진행중')



            # 취업률 (고용보험가입 6개월) 취업인원/수료인원
            series_count = len(soup.select('a.tool_tip_btn'))
            # count = len(soup.select('a.tool_tip_btn')) + 1
            cnt = 0
            for i in soup.select('table.view'):
                if i.text.find('취업률 고용보험가입 6개월') != -1:
                    for j in i.select('b'):
                        cnt += 1
                        if cnt % 3 == 2:
                            rate_empl = j.text.replace('\n','').replace('\t','')
                            empl_rate_6.append(rate_empl[:rate_empl.find("%")])
                            empl_6.append(rate_empl[rate_empl.find("%")+2:rate_empl.find("/")])
                            comp_6.append(rate_empl[rate_empl.find("/")+1:-1])

            while len(empl_rate_6) != series_count:
                empl_rate_6.insert(0,'훈련진행중')
            while len(empl_6) != series_count:
                empl_6.insert(0,'훈련진행중')
            while len(comp_6) != series_count:
                comp_6.insert(0,'훈련진행중')

            # 회차별 수강확정인원, 수강신청인원
            print(f'회차별 수강 [확정,신청] 인원')
            for a in soup.select('table.view'):
                for b in a.select('td'):
                    if b.text.find('수강확정인원') != -1:
            #             print(re.findall('\d+',b.text))
                        conf.append(re.findall('\d+',b.text)[0])
                        appl.append(re.findall('\d+',b.text)[1])

            count = len(soup.select('a.tool_tip_btn')) + 1

            for index, pat in enumerate(soup.select('a.tool_tip_btn')):
                series_url = f'https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId={pattern[0]}&tracseTme={index+1}&crseTracseSe={pattern[3]}&trainstCstmrId={pattern[2]}#undefined'
                resp_series = requests.get(series_url)
                soup_series = BeautifulSoup(resp_series.text, "html.parser")

                count -= 1
                print(f'{count} 회차 수집중...')
                print(f'링크 : {series_url}')

                # 차단 방지 시간 딜레이
                seed = np.random.randint(100)
                np.random.seed(seed)
                a = np.random.randint(5)
                time.sleep(a)

                # 기관명
                for organ in soup_series.select('p.add'):
                    orga.append(organ.text.strip())

                # 과정명
                for course in soup_series.select('h4.tit'):
            #         print(course.text.replace('\n','').replace('\t',''))
                    cour.append(course.text.replace('\n','').replace('\t',''))

                # 훈련비
                pric.append(soup_series.select('p.price')[0].text[:soup.select('p.price')[0].text.find(' 원')].replace('\n', '').replace('\t', ''))
            #     for price in soup_series.select('p.price'):
            #         print(price.text.strip().replace('\n','')[:price.text.find('원')].strip().replace('\t',''))
            #         pric.append(price.text.strip().replace('\n','')[:price.text.find('원')].strip().replace('\t',''))

                # 훈련시간
                for i in soup_series.select('span.con'):
                    if i.text.find('일') != -1 and i.text.find('시간') != -1:
            #             print(i.text.replace('\n','').replace('\t',''))
#                         spen.append(re.findall('\d+[ㄱ-ㅎ|가-힣|,]+\d+[ㄱ-ㅎ|가-힣|,]+',i.text))
                        spen.append(i.text.replace('\n,','').replace('\t','').strip().replace('\n',''))
                # NCS 자격분류
                for NCS in soup_series.select('span.con'):
                    if re.findall('[ㄱ-ㅎ|가-힣|A-Za-z]+\(\d+\)',NCS.text) != []:
            #             print(re.findall('.+\d{6,}.',NCS.text).pop().replace('\t',''))
                        NCS_dist.append(re.findall('[ㄱ-ㅎ|가-힣|A-Za-z]+\(\d+\)',NCS.text).pop().replace('\t',''))
                # 수강생 평균나이
#                 for g in soup.select('span.con'):
#                     if g.text.find('세') != -1:
#             #         print(re.findall("\d+\s세", g.text).pop().replace(' 세',''))
#                         mean_age.append(re.findall("\d+\s세", g.text).pop().replace(' 세',''))
#                 if mean_age == []:
#                     mean_age.append('-')
                inspect_age = []
                for g in soup_series.select('span.con'):

                    if re.findall('\d+\s세',g.text) == []:
                        inspect_age.append(int(-1))
                    if re.findall("\d+\s세", g.text) != []:
#         print(re.findall("\d+\s세", g.text).pop().replace(' 세',''))
                        mean_age.append(re.findall("\d+\s세", g.text).pop().replace(' 세',''))

                if sum(inspect_age) + len(soup.select('span.con')) == 0:
                    mean_age.append('-')

            print(orga)
            print(f'size : {len(orga)}')
            print(cour)
            print(f'size : {len(cour)}')
            print(pric)
            print(f'size : {len(pric)}')
            print(spen)
            print(f'size : {len(spen)}')
            print(NCS_dist)
            print(f'size : {len(NCS_dist)}')

            li_list =[orga,cour,seri,pric,prev,fini,spen,NCS_dist,recu,conf,appl,mean_age,sati,empl_rate_3,empl_3,comp_3,empl_rate_6,empl_6]
            var_list = ['orga','cour','seri','pric','prev','fini','spen','NCS_dist','recu','conf','appl','mean_age','sati','empl_rate_3','empl_3','comp_3','empl_rate_6','empl_6']
            
            dic = {}
            for li, cols in zip(li_list, df.columns.tolist()):
                dic[cols] = li
            df = pd.concat([df,pd.DataFrame(dic)], ignore_index=True)
df

1 크롤링 ...
1 번째 아이템 찾는 중...
링크 : https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20220000366834&tracseTme=1&trainstCstmrId=500020013014&crseTracseSe=C0061https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20220000366834&pageId=
--------------- 회차별 만족도 ---------------
1회차 만족도 수집중...
--------------- 회차별 훈련기간 ---------------
1회차 훈련 시작일, 종료일 수집중...
--------------- 회차별 모집인원 ---------------
1회차 모집인원 수집중...
회차별 수강 [확정,신청] 인원
1 회차 수집중...
링크 : https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20220000366834&tracseTme=1&crseTracseSe=C0061&trainstCstmrId=500020013014#undefined
['중부대학교(일학습병행)']
size : 1
['빅데이터 분석 및 플랫폼 구축 실무 양성과정모집중']
size : 1
['8,759,320']
size : 1
['115일,총920시간']
size : 1
['빅데이터분석(20010105)']
size : 1
2 번째 아이템 찾는 중...
링크 : https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20220000366893&tracseTme=2&trainstCstmrId=500020028562&crseTracseSe=C0061https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20220000366893&pageId=
----

7 회차 수집중...
링크 : https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20200000313547&tracseTme=15&crseTracseSe=C0061&trainstCstmrId=500020039927#undefined
6 회차 수집중...
링크 : https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20200000313547&tracseTme=16&crseTracseSe=C0061&trainstCstmrId=500020039927#undefined
5 회차 수집중...
링크 : https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20200000313547&tracseTme=17&crseTracseSe=C0061&trainstCstmrId=500020039927#undefined
4 회차 수집중...
링크 : https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20200000313547&tracseTme=18&crseTracseSe=C0061&trainstCstmrId=500020039927#undefined
3 회차 수집중...
링크 : https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20200000313547&tracseTme=19&crseTracseSe=C0061&trainstCstmrId=500020039927#undefined
2 회차 수집중...
링크 : https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20200000313547&tracseTme=20&crseTracseSe=C0061&trainstCstmrId=500020039927#undefined
1 회차 수집중...
링크 :

3 회차 수집중...
링크 : https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20210000313685&tracseTme=8&crseTracseSe=C0061&trainstCstmrId=500020013370#undefined
2 회차 수집중...
링크 : https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20210000313685&tracseTme=9&crseTracseSe=C0061&trainstCstmrId=500020013370#undefined
1 회차 수집중...
링크 : https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20210000313685&tracseTme=10&crseTracseSe=C0061&trainstCstmrId=500020013370#undefined
['하이미디어IT융합인재개발원', '하이미디어IT융합인재개발원', '하이미디어IT융합인재개발원', '하이미디어IT융합인재개발원', '하이미디어IT융합인재개발원', '하이미디어IT융합인재개발원', '하이미디어IT융합인재개발원', '하이미디어IT융합인재개발원', '하이미디어IT융합인재개발원', '하이미디어IT융합인재개발원']
size : 10
['AI&빅데이터기반 RPA융합 디지털트랜스포메이션시스템개발 모집마감', 'AI&빅데이터기반 RPA융합 디지털트랜스포메이션시스템개발 모집마감', 'AI&빅데이터기반 RPA융합 디지털트랜스포메이션시스템개발 모집마감', 'AI&빅데이터기반 RPA융합 디지털트랜스포메이션시스템개발 모집마감', 'AI&빅데이터기반 RPA융합 디지털트랜스포메이션시스템개발 모집중', 'AI&빅데이터기반 RPA융합 디지털트랜스포메이션시스템개발 모집중', 'AI&빅데이터기반 RPA융합 디지털트랜스포메이션시스템개발 모집중', 'AI&빅데이터기반 RPA융합 디지털트랜스포메이션시스템개발 모집중', 'A

--------------- 회차별 만족도 ---------------
1회차 만족도 수집중...
--------------- 회차별 훈련기간 ---------------
1회차 훈련 시작일, 종료일 수집중...
--------------- 회차별 모집인원 ---------------
1회차 : 모집인원 수집중...
회차별 수강 [확정,신청] 인원
1 회차 수집중...
링크 : https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20220000366776&tracseTme=1&crseTracseSe=C0061&trainstCstmrId=500020040446#undefined
['(학교법인)인하대학교']
size : 1
['스마트시티 실현을 위한 BIM 설계 인력양성모집중']
size : 1
['12,606,200']
size : 1
['184일,총1472시간']
size : 1
['데이터아키텍처(20010211)']
size : 1
13 번째 아이템 찾는 중...
링크 : https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20210000328538&tracseTme=3&trainstCstmrId=500020013381&crseTracseSe=C0061https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20210000328538&pageId=
--------------- 회차별 만족도 ---------------
3회차 만족도 수집중...
2회차 만족도 수집중...
1회차 만족도 수집중...
--------------- 회차별 훈련기간 ---------------
3회차 훈련 시작일, 종료일 수집중...
2회차 훈련 시작일, 종료일 수집중...
1회차 훈련 시작일, 종료일 수집중...
--------------- 회차별 모집인원 ---------------
3회차 : 모집인원 수집중.

--------------- 회차별 만족도 ---------------
2회차 만족도 수집중...
1회차 만족도 수집중...
--------------- 회차별 훈련기간 ---------------
2회차 훈련 시작일, 종료일 수집중...
1회차 훈련 시작일, 종료일 수집중...
--------------- 회차별 모집인원 ---------------
2회차 : 모집인원 수집중...
1회차 모집인원 수집중...
회차별 수강 [확정,신청] 인원
2 회차 수집중...
링크 : https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20220000366724&tracseTme=1&crseTracseSe=C0061&trainstCstmrId=500020033667#undefined
1 회차 수집중...
링크 : https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20220000366724&tracseTme=2&crseTracseSe=C0061&trainstCstmrId=500020033667#undefined
['한울직업전문학교', '한울직업전문학교']
size : 2
['[PBL]스마트 웹&앱 콘텐츠 전문개발자 양성모집마감', '[PBL]스마트 웹&앱 콘텐츠 전문개발자 양성모집중']
size : 2
['7,536,320', '7,536,320']
size : 2
['111일,총880시간', '111일,총880시간']
size : 2
['UX엔지니어링(20010207)', 'UX엔지니어링(20010207)']
size : 2
19 번째 아이템 찾는 중...
링크 : https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20210000328552&tracseTme=2&trainstCstmrId=500020041301&crseTracseSe=C0061https://www.hrd.go.kr/hrdp/co

...

--------------- 회차별 만족도 ---------------
3회차 만족도 수집중...
2회차 만족도 수집중...
1회차 만족도 수집중...
--------------- 회차별 훈련기간 ---------------
3회차 훈련 시작일, 종료일 수집중...
2회차 훈련 시작일, 종료일 수집중...
1회차 훈련 시작일, 종료일 수집중...
--------------- 회차별 모집인원 ---------------
3회차 : 모집인원 수집중...
2회차 모집인원 수집중...
1회차 모집인원 수집중...
회차별 수강 [확정,신청] 인원
3 회차 수집중...
링크 : https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20220000366918&tracseTme=1&crseTracseSe=C0061&trainstCstmrId=500020060638#undefined
2 회차 수집중...
링크 : https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20220000366918&tracseTme=2&crseTracseSe=C0061&trainstCstmrId=500020060638#undefined
1 회차 수집중...
링크 : https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20220000366918&tracseTme=3&crseTracseSe=C0061&trainstCstmrId=500020060638#undefined
['(주)문화방송', '(주)문화방송', '(주)문화방송']
size : 3
['3D 증강&확장현실(AR&XR) 디자이너 양성 과정모집마감', '3D 증강&확장현실(AR&XR) 디자이너 양성 과정모집중', '3D 증강&확장현실(AR&XR) 디자이너 양성 과정모집중']
size : 3
['17,424,000', '17,424,000', '17,424,000']
size 

1 회차 수집중...
링크 : https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20220000366825&tracseTme=2&crseTracseSe=C0061&trainstCstmrId=500020050800#undefined
['이젠아카데미평생교육원', '이젠아카데미평생교육원']
size : 2
['AWS 클라우드환경 네이티브 앱 개발 & Devops모집마감', 'AWS 클라우드환경 네이티브 앱 개발 & Devops모집중']
size : 2
['7,365,040', '7,365,040']
size : 2
['147일,총860시간', '144일,총860시간']
size : 2
['응용SW엔지니어링(20010202)', '응용SW엔지니어링(20010202)']
size : 2
13 크롤링 ...
121 번째 아이템 찾는 중...
링크 : https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20210000320939&tracseTme=4&trainstCstmrId=500020019318&crseTracseSe=C0061https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20210000320939&pageId=
--------------- 회차별 만족도 ---------------
4회차 만족도 수집중...
3회차 만족도 수집중...
2회차 만족도 수집중...
1회차 만족도 수집중...
--------------- 회차별 훈련기간 ---------------
4회차 훈련 시작일, 종료일 수집중...
3회차 훈련 시작일, 종료일 수집중...
2회차 훈련 시작일, 종료일 수집중...
1회차 훈련 시작일, 종료일 수집중...
--------------- 회차별 모집인원 ---------------
4회차 : 모집인원 수집중...
3회차 모집인원 수집중...
2회차 : 모집인원 수집중...
1

--------------- 회차별 만족도 ---------------
2회차 만족도 수집중...
1회차 만족도 수집중...
--------------- 회차별 훈련기간 ---------------
2회차 훈련 시작일, 종료일 수집중...
1회차 훈련 시작일, 종료일 수집중...
--------------- 회차별 모집인원 ---------------
2회차 : 모집인원 수집중...
1회차 : 모집인원 수집중...
취업률 수집 완중...
회차별 수강 [확정,신청] 인원
2 회차 수집중...
링크 : https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20210000328760&tracseTme=1&crseTracseSe=C0061&trainstCstmrId=500033820700#undefined
1 회차 수집중...
링크 : https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20210000328760&tracseTme=2&crseTracseSe=C0061&trainstCstmrId=500033820700#undefined
['코리아아이티신촌학원', '코리아아이티신촌학원']
size : 2
['스마트팩토리 MES 구축 전문가 양성과정모집마감', '스마트팩토리 MES 구축 전문가 양성과정모집중']
size : 2
['6,474,380', '6,474,380']
size : 2
['95일,총756시간', '95일,총756시간']
size : 2
['응용SW엔지니어링(20010202)', '응용SW엔지니어링(20010202)']
size : 2
127 번째 아이템 찾는 중...
링크 : https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20210000328853&tracseTme=3&trainstCstmrId=500034091717&crseTracseSe=C0061https://www.h

--------------- 회차별 만족도 ---------------
2회차 만족도 수집중...
1회차 만족도 수집중...
--------------- 회차별 훈련기간 ---------------
2회차 훈련 시작일, 종료일 수집중...
1회차 훈련 시작일, 종료일 수집중...
--------------- 회차별 모집인원 ---------------
2회차 : 모집인원 수집중...
1회차 : 모집인원 수집중...
취업률 수집 완중...
회차별 수강 [확정,신청] 인원
2 회차 수집중...
링크 : https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20210000313599&tracseTme=1&crseTracseSe=C0061&trainstCstmrId=500020048147#undefined
1 회차 수집중...
링크 : https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20210000313599&tracseTme=2&crseTracseSe=C0061&trainstCstmrId=500020048147#undefined
['한국IT교육원', '한국IT교육원']
size : 2
['5G·AI 자율주행 인력양성(feat.실증도로)모집마감', '5G·AI 자율주행 인력양성(feat.실증도로)모집중']
size : 2
['9,111,590', '9,111,590']
size : 2
['122일,총957시간', '122일,총957시간']
size : 2
['정보기술전략(20010101)', '정보기술전략(20010101)']
size : 2
135 번째 아이템 찾는 중...
링크 : https://www.hrd.go.kr/hrdp/co/pcobo/PCOBO0100P.do?tracseId=AIG20220000366722&tracseTme=1&trainstCstmrId=500020031803&crseTracseSe=C0061https://www.hrd

,기관명,과정명,회차,훈련비,훈련시작일,훈련종료일,훈련시간,NCS 직무분류,모집인원,수강확정인원,수강신청인원,수강생 평균 연령대(세),수강생 만족도,취업률 고용보험가입 3개월,취업인원(3개월),수료인원,취업률 고용보험가입 6개월,취업인원(6개월)
0,중부대학교(일학습병행),빅데이터 분석 및 플랫폼 구축 실무 양성과정모집중,1,"8,759,320",2022-08-01,2023-01-06,"115일,총920시간",빅데이터분석(20010105),훈련진행중,18,21,23,훈련진행중,훈련진행중,훈련진행중,훈련진행중,훈련진행중,훈련진행중
1,(주)포스코 미래창조아카데미,웹개발자 입문 과정모집마감,2,"6,534,000",2022-08-01,2022-10-31,"61일,총360시간",응용SW엔지니어링(20010202),훈련진행중,26,26,25,훈련진행중,훈련진행중,훈련진행중,훈련진행중,훈련진행중,훈련진행중
2,(주)포스코 미래창조아카데미,웹개발자 입문 과정모집중,1,"6,534,000",2022-06-15,2022-09-14,"61일,총360시간",응용SW엔지니어링(20010202),훈련진행중,20,20,27,훈련진행중,훈련진행중,훈련진행중,훈련진행중,훈련진행중,훈련진행중
3,방송정보국제교육원,3D 기반 실감형(VR/AR) 콘텐츠 제작 전문가모집마감,6,"7,011,840",2022-08-01,2023-01-25,"122일,총960시간",VR콘텐츠디자인(08020112),훈련진행중,0,0,27,훈련진행중,훈련진행중,훈련진행중,훈련진행중,훈련진행중,훈련진행중
4,방송정보국제교육원,3D 기반 실감형(VR/AR) 콘텐츠 제작 전문가모집마감,5,"7,011,840",2022-08-24,2023-02-22,"123일,총960시간",VR콘텐츠디자인(08020112),0,0,12,27,0,훈련진행중,훈련진행중,훈련진행중,훈련진행중,훈련진행중
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,한국정보교육원(구.경원직업전문학교),웹 프론트엔드 클라우드 콘솔 개발자 양성모집중,1,"8,426,970",2022-10-24,2023-04-17,"123일,총984시간",응용SW엔지니어링(20010202),0,0,0,-,0,훈련진행중,훈련진행중,훈련진행중,훈련진행중,훈련진행중
526,글로벌산업기술교육원,"로봇설계를 통한 스마트제조 가상화 구현(CATIA, VR)모집마감",4,"6,219,200",2022-11-24,2023-04-17,"100일,총800시간",기계시스템설계(15010202),0,0,0,27,0,훈련진행중,훈련진행중,훈련진행중,훈련진행중,훈련진행중
527,글로벌산업기술교육원,"로봇설계를 통한 스마트제조 가상화 구현(CATIA, VR)모집마감",3,"6,219,200",2022-10-27,2023-03-20,"100일,총800시간",기계시스템설계(15010202),0,0,0,26,0,훈련진행중,훈련진행중,훈련진행중,훈련진행중,훈련진행중
528,글로벌산업기술교육원,"로봇설계를 통한 스마트제조 가상화 구현(CATIA, VR)모집중",2,"6,219,200",2022-06-17,2022-11-10,"100일,총800시간",기계시스템설계(15010202),훈련진행중,12,12,-,훈련진행중,훈련진행중,훈련진행중,훈련진행중,훈련진행중,훈련진행중


In [497]:
df.to_excel('HRDcrawling.xlsx', index=False)